In [1]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        pass
    def forward(self, x): 
        pass

mynet = MyNet()


In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
to_tensor = ToTensor() # img -> tensor
to_pil = ToPILImage()
lena = Image.open('data/captcha/test/0011.jpg')
dir(to_tensor)

In [29]:
##### import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


#data = dset.MNIST('data/mnist', train=True, transform=None, target_transform=None, download=True)
has_cuda = False
kwargs = {'num_workers': 1, 'pin_memory': True} if has_cuda else {}
"""加载数据。组合数据集和采样器，提供数据上的单或多进程迭代器
参数：
dataset：Dataset类型，从其中加载数据
batch_size：int，可选。每个batch加载多少样本
shuffle：bool，可选。为True时表示每个epoch都对数据进行洗牌
sampler：Sampler，可选。从数据集中采样样本的方法。
num_workers：int，可选。加载数据时使用多少子进程。默认值为0，表示在主进程中加载数据。
collate_fn：callable，可选。
pin_memory：bool，可选
drop_last：bool，可选。True表示如果最后剩下不完全的batch,丢弃。False表示不丢弃。
"""
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/mnist', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
batch_size=1000, shuffle=True, **kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()    
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()#把module设成training模式，对Dropout和BatchNorm有影响
    for batch_idx, (data, target) in enumerate(train_loader):
        if has_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)#Variable类对Tensor对象进行封装，会保存该张量对应的梯度，以及对生成该张量的函数grad_fn的一个引用。如果该张量是用户创建的，grad_fn是None，称这样的Variable为叶子Variable。
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)#负log似然损失
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))



def test(epoch):
    model.eval()#把module设置为评估模式，只对Dropout和BatchNorm模块有影响
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if has_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]#Variable.data
        pred = output.data.max(1)[1] # get the index of the max log-probability
        print(output.data)
        correct += pred.eq(target.data).cpu().sum()
        #print(target.data)
    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 11):
    train(epoch)
    print("训练完成，进行测试")
    test(epoch)
    print("测试结束，下一轮训练开始 %d" % epoch)
print("运行结束")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322766
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.288310
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.287744
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.260727
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.242691
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.210801
训练完成，进行测试

-2.3567 -2.3337 -2.3074  ...  -2.1336 -2.3599 -2.2155
-2.0767 -2.6192 -2.3753  ...  -2.1788 -2.4483 -2.1492
-2.4347 -2.2286 -2.3511  ...  -2.1244 -2.2891 -2.1608
          ...             ⋱             ...          
-1.8040 -2.7960 -2.3643  ...  -2.1220 -2.3391 -2.1002
-2.1847 -2.5813 -2.3639  ...  -2.2472 -2.3382 -2.1576
-1.9976 -2.5589 -2.4123  ...  -2.2179 -2.3060 -2.1754
[torch.FloatTensor of size 1000x10]


-1.9674 -2.6563 -2.3437  ...  -2.1669 -2.4024 -2.1087
-2.4820 -2.2711 -2.3062  ...  -2.1766 -2.2134 -2.3038
-2.3929 -2.4075 -2.3283  ...  -2.2269 -2.1229 -2.1425
          ...             ⋱             ...          
-2.2940 -2.5372 -2.3232  ...  -2.1916 -2.1852 -2.2179
-2.6329 


-1.7844 -4.2823 -2.6672  ...  -2.0577 -2.9114 -1.8884
-1.5218 -4.4793 -2.6784  ...  -2.1896 -3.0338 -1.9504
-3.0297 -2.0384 -2.3893  ...  -2.6374 -2.1502 -2.2937
          ...             ⋱             ...          
-4.7158 -0.5663 -2.9037  ...  -2.5471 -2.4721 -3.0696
-2.8516 -3.4670 -2.6754  ...  -3.0048 -2.5486 -2.2879
-0.8610 -4.7225 -3.1622  ...  -2.4795 -2.7525 -2.3462
[torch.FloatTensor of size 1000x10]


Test set: Average loss: 1.4487, Accuracy: 7163/10000 (72%)

测试结束，下一轮训练开始
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.678862
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.591485
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.478549
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.398495
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.279861
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.221920
训练完成，进行测试

 -3.1511  -2.9177  -2.3709  ...   -3.1531  -0.7617  -3.2653
 -4.5119  -6.7904  -4.1211  ...   -6.2018  -4.6569  -3.8808
 -0.3761  -6.4414  -3.7995  ...   -2.2881  -4.0792  -2.9985
           ...          


 -2.8426  -2.1030  -1.3211  ...   -1.7428  -1.5855  -3.4543
 -4.9512  -3.1844  -0.3687  ...   -3.1396  -1.6917  -6.4137
 -7.9065  -8.0039  -7.5590  ...   -3.9891  -5.6059  -1.2912
           ...               ⋱              ...            
 -3.3587  -3.7332  -4.2883  ...   -1.4869  -2.4304  -1.3353
 -6.0930  -9.8361  -7.3743  ...   -6.3412  -5.8334  -2.7031
 -7.4156  -9.7960  -6.8009  ...  -11.3580  -6.2833  -7.4693
[torch.FloatTensor of size 1000x10]


 -1.0184  -5.4010  -2.0687  ...   -2.5562  -3.9123  -1.8955
 -8.0692  -7.4434  -6.6108  ...   -5.8214  -5.4619  -2.1930
 -8.0627  -4.9801  -3.3590  ...   -4.8860  -4.3945  -5.1269
           ...               ⋱              ...            
 -6.9800  -9.3208  -0.0065  ...   -7.0942  -6.7162  -8.2276
 -5.3179  -5.2430  -6.5830  ...   -1.1523  -3.1685  -0.7171
 -6.1857  -3.7417  -6.2177  ...   -2.7312  -2.3700  -0.7664
[torch.FloatTensor of size 1000x10]


 -6.1623  -5.7765  -0.0644  ...   -5.4421  -3.9920  -6.1332
 -5.6296  -1.9699  -3.2

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.846517
Train Epoch: 6 [10000/60000 (17%)]	Loss: 0.793942
Train Epoch: 6 [20000/60000 (33%)]	Loss: 0.758842
Train Epoch: 6 [30000/60000 (50%)]	Loss: 0.798550
Train Epoch: 6 [40000/60000 (67%)]	Loss: 0.732543
Train Epoch: 6 [50000/60000 (83%)]	Loss: 0.820268
训练完成，进行测试

-2.9848e+00 -7.1160e+00 -2.9486e+00  ...  -5.9146e+00 -2.5472e-01 -5.3252e+00
-6.2847e+00 -9.3711e+00 -2.6916e-02  ...  -7.4155e+00 -6.7469e+00 -6.9542e+00
-7.9181e+00 -8.3093e+00 -3.9015e+00  ...  -9.7736e+00 -6.1425e+00 -7.1008e+00
                ...                   ⋱                   ...                
-2.6277e-01 -1.1835e+01 -6.7471e+00  ...  -8.7672e+00 -6.5205e+00 -7.0636e+00
-1.2509e+01 -6.9550e-03 -7.5230e+00  ...  -6.0180e+00 -6.0067e+00 -7.9395e+00
-5.2716e+00 -5.0511e+00 -3.3383e+00  ...  -5.9630e+00 -1.8234e+00 -3.9648e+00
[torch.FloatTensor of size 1000x10]


-8.9427e+00 -5.6808e-02 -4.4989e+00  ...  -4.3235e+00 -3.8935e+00 -6.1972e+00
-1.1136e+01 -1.3878e-02 -7.2136e


-1.0583e+01 -1.2176e+01 -1.2897e+01  ...  -3.2238e+00 -5.1156e+00 -1.4866e-01
-4.7427e+00 -3.8454e+00 -2.0748e+00  ...  -3.6955e+00 -2.3591e+00 -3.5218e+00
-8.8997e+00 -1.1171e+01 -9.3117e+00  ...  -2.0054e-02 -8.5988e+00 -4.0054e+00
                ...                   ⋱                   ...                
-1.0165e+01 -9.4607e+00 -7.6375e+00  ...  -5.1426e+00 -6.4086e+00 -9.6123e-01
-9.9576e+00 -9.6993e+00 -1.0153e+01  ...  -2.6832e+00 -6.5759e+00 -2.5869e-01
-1.1987e+01 -1.2420e+01 -1.1000e+01  ...  -6.6072e+00 -8.9263e+00 -2.1690e+00
[torch.FloatTensor of size 1000x10]


-7.9705e+00 -9.1998e+00 -5.9204e+00  ...  -1.2559e+01 -8.0745e+00 -9.3765e+00
-3.4839e-03 -1.5771e+01 -1.2779e+01  ...  -8.4798e+00 -9.3262e+00 -1.1058e+01
-7.8534e+00 -1.6000e+01 -1.4617e+01  ...  -5.8302e-02 -1.0711e+01 -2.8824e+00
                ...                   ⋱                   ...                
-1.4791e+01 -1.2252e+01 -1.2828e+01  ...  -7.4805e+00 -7.8696e+00 -2.0801e+00
-9.5215e+00 -4.7432e-02 -


-1.1602e+01 -5.9548e+00 -5.0887e+00  ...  -7.9104e+00 -5.1014e+00 -6.9112e+00
-6.6215e+00 -7.6396e+00 -5.0701e+00  ...  -9.0432e+00 -6.0335e-02 -6.5740e+00
-8.9136e+00 -1.5558e+01 -1.3353e+01  ...  -2.1616e-02 -1.0845e+01 -3.8711e+00
                ...                   ⋱                   ...                
-1.1299e+01 -6.9296e+00 -9.6843e+00  ...  -1.8873e+00 -4.2584e+00 -2.4770e-01
-1.0142e+01 -8.3509e+00 -8.5703e+00  ...  -3.7704e+00 -5.8823e+00 -3.3145e-01
-2.5651e+00 -2.8776e+00 -3.6173e+00  ...  -3.3763e+00 -2.9358e+00 -2.8757e+00
[torch.FloatTensor of size 1000x10]


-6.9238e+00 -1.0882e+01 -9.6918e+00  ...  -8.9652e+00 -3.8921e+00 -5.0428e+00
-5.8173e+00 -8.0992e+00 -8.7175e+00  ...  -4.8895e+00 -3.1743e+00 -3.8456e+00
-1.0837e+01 -4.1424e-02 -6.4342e+00  ...  -3.8673e+00 -4.5485e+00 -6.2624e+00
                ...                   ⋱                   ...                
-7.5198e-03 -1.0618e+01 -5.8479e+00  ...  -6.8439e+00 -7.4902e+00 -8.3712e+00
-7.6478e+00 -1.2711e+01 -

Train Epoch: 10 [30000/60000 (50%)]	Loss: 0.530349
Train Epoch: 10 [40000/60000 (67%)]	Loss: 0.552887
Train Epoch: 10 [50000/60000 (83%)]	Loss: 0.494900
训练完成，进行测试

-1.4907e+01 -8.0617e+00 -7.2873e+00  ...  -9.2135e+00 -8.4780e+00 -8.9335e+00
-5.8533e+00 -9.6532e+00 -4.6224e+00  ...  -9.8729e+00 -3.3874e-02 -6.9778e+00
-1.0885e+01 -9.2004e+00 -7.7931e+00  ...  -6.2415e+00 -6.1275e+00 -6.9888e+00
                ...                   ⋱                   ...                
-1.1213e+01 -1.3483e-02 -6.1936e+00  ...  -6.7758e+00 -4.7234e+00 -8.6021e+00
-1.4012e+01 -7.7672e+00 -1.0459e+01  ...  -7.6562e+00 -4.6467e+00 -1.2719e+00
-9.9629e+00 -7.3779e+00 -9.1210e+00  ...  -9.1654e-01 -4.9612e+00 -5.4609e-01
[torch.FloatTensor of size 1000x10]


-5.2680e+00 -6.5524e+00 -5.8670e+00  ...  -8.7235e+00 -6.2498e-01 -5.2407e+00
-6.8640e+00 -6.7766e+00 -6.7258e+00  ...  -4.6573e+00 -1.6179e+00 -2.7115e+00
-9.6126e+00 -8.3902e+00 -7.9318e+00  ...  -4.7922e+00 -6.7081e+00 -1.0057e+00
                ..

In [11]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(train_loader[batch_idx])
    print()
    print(target)
    break;

TypeError: 'DataLoader' object does not support indexing